In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
from itertools import combinations

import os, sys, warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    # Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
                    
    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    return df

In [3]:
def gen_v1_features(df, prices):

    # V1 features: directly apply formula to a single row
    v1_features = {
        "volume": "ask_size + bid_size",
        "mid_price": "(ask_price + bid_price)/2",
        "liquidity_imbalance": "(bid_size-ask_size)/(bid_size+ask_size)",
        "matched_imbalance": "(imbalance_size - matched_size)/(matched_size+imbalance_size)",
        "size_imbalance": "bid_size / ask_size",
        "imbalance_intensity": "imbalance_size / volume",
        "matched_intensity": "matched_size / volume",
        "price_spread": "ask_price - bid_price",
        'market_urgency': 'price_spread * liquidity_imbalance',
        'depth_pressure': '(ask_size - bid_size) * (far_price - near_price)',
        'price_pressure': 'imbalance_size * (ask_price - bid_price)',
        'imbalance_with_flag': 'imbalance_size * imbalance_buy_sell_flag',
        'minute': 'seconds_in_bucket // 60',
    }

    # include pair-wise price imbalances
    for c in combinations(prices, 2):
        v1_features[f"{c[0]}_{c[1]}_imbalance"] = f"({c[0]} - {c[1]}) / ({c[0]} + {c[1]})"
    
    for k, v in v1_features.items():
        df[k] = df.eval(v)
        
    v1_feature_category = {
        'imb_buy_side': "(imbalance_buy_sell_flag == 1)",
        'imb_sell_side': "(imbalance_buy_sell_flag == -1)",
        'first_half_session': '(seconds_in_bucket <= 240)',
        'second_half_session': '(seconds_in_bucket > 240)'
    }
    
    for k, v in v1_feature_category.items():
        df[k] = df.eval(v).astype(np.int8)
        
    df = reduce_mem_usage(df, verbose=0)
        
    return df, list(v1_features.keys()), list(v1_feature_category.keys())
        
    

In [4]:
def gen_v2_features(df, v2_feat_cols):
    
    # V2 features: cross-section features
    # V2 features are generated on the groupby(['date_id', 'seconds_in_bucket'])
    # These features includes:
    # 1. statistics of V1 features (non-categorical)
    # 2. rank of V1 features for each stocks (non-categorical)
    
    group = df.groupby(['date_id', 'seconds_in_bucket'])

    v2_features_stats = ['mean', 'median', 'std', 'min', 'max']

    # calculate statistics of V1 features for each stock
    df_v2 = group[v2_feat_cols].agg(v2_features_stats).reset_index()
    df_v2.columns = ['date_id', 'seconds_in_bucket'] + [f"{c[1]}_{c[0]}" for c in df_v2.columns[2:]]
    df = df.merge(df_v2, on=['date_id', 'seconds_in_bucket'], how='left')
    

    # calculate rank of V1 features for each stock
    df_v2 = group[v2_feat_cols].rank(pct=True).add_prefix('rank_')
    df = df.merge(df_v2, left_index=True, right_index=True, how='left')
    
    df = reduce_mem_usage(df, verbose=0)
    
    v2_features =\
        [f"{s}_{c}" for c in v2_feat_cols for s in v2_features_stats] + \
        [f"rank_{c}" for c in v2_feat_cols]
        
    return df, v2_features
    

In [5]:
def gen_v3_features(df, prices, sizes, v1_features):
    # V3 features: rolling statistics of V1 features (non-categorical)
    # V3 features are generated on the groupby(['date_id', 'stock_id'])
    # here we introduce ta-lib functions to calculate TA indicators

    # V3.1 relative change of V1 features by shift(1)
    # for prices, we calculate the change in basis points (*1e4)
    # for other features, we calculate the change in percentage (*1e2)
    group_by_stock = df.groupby(['date_id', 'stock_id'])
    
    relative_price = group_by_stock[prices].pct_change(1).add_prefix('pct_')*1e4
    relative_others = group_by_stock[sizes+v1_features].pct_change(1).add_prefix('pct_')*1e2

    df = pd.concat([df, relative_price, relative_others], axis=1)
    v3_features = list(relative_price.columns) + list(relative_others.columns)
    
    # V3.2 Simple TA indicators
    # Those are simple TA indicators that use only one feature
    df_v3 = group_by_stock[prices + sizes + v1_features].rolling(5).agg(['mean', 'std', 'max', 'min']).reset_index()
    stats_cols = [f"{c[1]}_{c[0]}_5" for c in df_v3.columns[2:]]
    df_v3.columns = ['date_id', 'stock_id'] + stats_cols
    df_v3.set_index('_level_2_5', inplace=True)
    df_v3.drop(columns=['date_id', 'stock_id'], inplace=True)
    
    df = df.merge(df_v3, left_index=True, right_index=True, how='left')
    v3_features += stats_cols
        
    # # V3.3 TA indicators that use multiple features
    def composite_ta(df):

        ad_osc = ta.ADOSC(df['ask_price'], df['bid_price'], df['wap'], df['volume'], fastperiod=3, slowperiod=5)
        macd, macdsignal, macdhist = ta.MACD(df['wap'], fastperiod=5, slowperiod=11, signalperiod=3)
        
        return pd.DataFrame({
            'ema': ta.EMA(df['wap'], timeperiod=5),
            'rsi': ta.RSI(df['wap'], timeperiod=5),
            'cci': ta.CCI(df['ask_price'], df['bid_price'], df['wap'], timeperiod=5),
            'mfi': ta.MFI(df['ask_price'], df['bid_price'], df['wap'], df['volume'], timeperiod=5),
            'ad_osc': ad_osc,
            'macd': macd,
            'macdsignal': macdsignal,
            'macdhist': macdhist
        })
    
    df_v3 = group_by_stock.apply(composite_ta) 
    v3_features += df_v3.columns.tolist()
    
    df_v3.reset_index(inplace=True)
    df_v3.set_index('level_2', inplace=True)
    df_v3.drop(columns=['date_id', 'stock_id'], inplace=True)
    
    df = pd.concat([df, df_v3], axis=1)
    
    return df, v3_features

In [6]:
df = pd.read_csv("/home/lishi/projects/Competition/data/train.csv")
df = df[~df['target'].isnull()] 

print(df.shape)
print(f"Trading days: {df['date_id'].nunique()}")
print(f"Stocks: {df['stock_id'].nunique()}")

df['far_price'] = df['far_price'].fillna(0)
df['near_price'] = df['near_price'].fillna(0)

df = reduce_mem_usage(df, verbose=1)

df.head()

(5237892, 17)
Trading days: 481
Stocks: 200
Memory usage of dataframe is 719.32 MB
Memory usage after optimization is: 344.67 MB
Decreased by 52.08%


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.180603e+06,1,0.999812,13380277.00,0.0,0.0,0.999812,60651.500000,1.000026,8493.030273,1.0,-3.029704,0,0_0_0
1,1,0,0,1.666039e+05,-1,0.999896,1642214.25,0.0,0.0,0.999896,3233.040039,1.000660,20605.089844,1.0,-5.519986,0,0_0_1
2,2,0,0,3.028799e+05,-1,0.999561,1819368.00,0.0,0.0,0.999403,37956.000000,1.000298,18995.000000,1.0,-8.389950,0,0_0_2
3,3,0,0,1.191768e+07,-1,1.000171,18389746.00,0.0,0.0,0.999999,2324.899902,1.000214,479032.406250,1.0,-4.010201,0,0_0_3
4,4,0,0,4.475500e+05,-1,0.999532,17860614.00,0.0,0.0,0.999394,16485.539062,1.000016,434.100006,1.0,-7.349849,0,0_0_4


In [7]:
prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

feature_cols = prices + sizes

df, v1_features, v1_feature_category = gen_v1_features(df, prices)
feature_cols += (v1_features+v1_feature_category)

df, v2_features = gen_v2_features(df, prices+sizes+v1_features)
feature_cols += v2_features

df, v3_features = gen_v3_features(df, prices, sizes, v1_features)
feature_cols += v3_features

df.fillna(0, inplace=True)
df = reduce_mem_usage(df, verbose=1)

print(len(feature_cols))

In [ ]:
df.to_csv("/home/lishi/projects/Competition/data/train_features.csv", index=False)

In [ ]:
len(feature_cols)